### Imports

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          LlamaForSequenceClassification,
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Eval Using Weights and Biases

In [2]:
# import wandb

# run = wandb.init(
#     project="Fine-Tune Llama for Medical Dataset",
#     job_type="training",
#     anonymous="allow"
# )

### Loading and processing the dataset

In [10]:
df = pd.read_csv("datasets/md-data-statement.csv")
df.head()

,Statement,NoShow
0,Patient Information:\n- Gender: Female\n- Age:...,No
1,Patient Information:\n- Gender: Male\n- Age: 5...,No
2,Patient Information:\n- Gender: Female\n- Age:...,No
3,Patient Information:\n- Gender: Female\n- Age:...,No
4,Patient Information:\n- Gender: Female\n- Age:...,No


In [11]:
yes_df = df[df['NoShow'] == 'Yes']
no_df = df[df['NoShow'] == 'No']

# min_count = min(len(yes_df), len(no_df), 100000)
min_count = min(len(yes_df), len(no_df), 10)

yes_sample = yes_df.sample(n=min_count, random_state=42)
no_sample = no_df.sample(n=min_count, random_state=42)

balanced_df = pd.concat([yes_sample, no_sample]).reset_index(drop=True)

df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

In [13]:
# Define the prompt generation functions
# def generate_prompt(data_point):
#     return f"""
#     Classify the text into Yes or No based on statement, and return answer as the corresponding label.
#     Text: {data_point["Statement"]}
#     Label: {data_point["NoShow"]}
#     """

# def generate_test_prompt(data_point):
#     return f"""
#     Classify the text into Yes or No based on statement, and return answer as the corresponding label.
#     Text: {data_point["Statement"]}
#     Label:
#     """

def generate_prompt(data_point):
    return f"""Patient Information: Gender, Age, Appointment Date, Scheduled For, Medical Conditions, Hypertension, Diabetes, Alcoholism, Handicap, SMS Reminder Sent. Based on the these information in the text, label whether the patient will show up for their appointment. Respond with either "Yes" (patient will not show up) or "No" (patient will show up) in after the label at the end of the response.
text: {data_point["Statement"]}
label: {data_point["NoShow"]}""".strip()

def generate_test_prompt(data_point):
    return f"""Patient Information: Gender, Age, Appointment Date, Scheduled For, Medical Conditions, Hypertension, Diabetes, Alcoholism, Handicap, SMS Reminder Sent. Based on the these information in the text, label whether the patient will show up for their appointment. Respond with either "Yes" (patient will not show up) or "No" (patient will show up) in after the label at the end of the response.
text: {data_point["Statement"]}
label: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'NoShow']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])



/usr/lib/python3/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [14]:
X_train.NoShow.value_counts()

Yes    8
No     8
Name: NoShow, dtype: int64

In [15]:
y_true.value_counts()

No     1
Yes    1
Name: NoShow, dtype: int64

### Convert to datasets

In [16]:

train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [17]:
train_data['text'][3]

'Patient Information: Gender, Age, Appointment Date, Scheduled For, Medical Conditions, Hypertension, Diabetes, Alcoholism, Handicap, SMS Reminder Sent. Based on the these information in the text, label whether the patient will show up for their appointment. Respond with either "Yes" (patient will not show up) or "No" (patient will show up) in after the label at the end of the response.\ntext: Patient Information:\n- Gender: Female\n- Age: 41\n- Appointment Date: 2016-05-04T13:36:30Z\n- Scheduled For: 2016-05-20T00:00:00Z\n- Medical Conditions:\n  * Hypertension: NO\n  * Diabetes: NO\n  * Alcoholism: NO\n- Handicap: NO\n- SMS Reminder Sent: NO\nlabel: Yes'

### Loading the model and tokenizer

In [18]:
# base_model_name = "meta-llama/Meta-Llama-3.1-8B"
base_model_name = "meta-llama/Meta-Llama-3.1-8B-instruct"


In [19]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)


model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id


In [21]:
pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=2, 
                        temperature=0.1)

### Model evalution before fine-tuning

In [22]:
def predict(test, model, tokenizer, pipe):
    y_pred = []
    categories = ["Yes", "No"]
    
    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        
        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")
    
    return y_pred

In [23]:
y_pred = predict(X_test, model, tokenizer, pipe)

100%|██████████| 2/2 [00:00<00:00,  2.05it/s]


In [24]:
def evaluate(y_true, y_pred):
    labels = ["Yes","No"]
    mapping = {label: idx for idx, label in enumerate(labels)}
    
    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data
    
    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [25]:
evaluate(y_true, y_pred)

Accuracy: 0.500
Accuracy for label Yes: 0.000
Accuracy for label No: 1.000

Classification Report:
              precision    recall  f1-score   support

         Yes       0.00      0.00      0.00         1
          No       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2


Confusion Matrix:
[[0 1]
 [0 1]]


/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Extracting the linear modules names

In [26]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [27]:
modules = find_all_linear_names(model)
modules

['k_proj', 'up_proj', 'o_proj', 'gate_proj', 'down_proj', 'q_proj', 'v_proj']

### Setting up the model

In [28]:
output_dir="llama-3.1-fine-tuned-model"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,                         
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    # report_to="wandb",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ubuntu/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:327: UserWarning: You passed a `dataset_kwargs` argument t

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

### Model Training

In [29]:
# Train model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
# wandb.finish()
# model.config.use_cache = True

### Saving the model and tokenizer

In [ ]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

NameError: name 'trainer' is not defined

### Testing model after fine-tuning 

In [ ]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

NameError: name 'predict' is not defined